<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/Pubquiz_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pubquiz-chatGPT für alle!

*v1.4* - Ein Notebook, mit dem man (mit einem gespendeten API-Token) GPT-4 nutzen kann, ohne sich anzumelden - und noch ein paar zusätzliche Einstellungen beeinflussen kann wie z.B. die Persönlichkeit, das verwendete Modell und die Parameter.

## Der Chatbot

Die Chatbot-Persönlichkeit "Harald" habe ich mir bei Claus Hesseling ausgeliehen.

Damit man diesen Chatbot nutzen kann, **muss man ein gültiges API-Token in das entsprechende Feld kopieren.**

Die Parameter:
- Die **Temperatur** bestimmt das Maß an Zufall, das das Sprachmodell nutzt - man könnte auch sagen: je höher die Temperatur, desto kreativer wird es.
- Das **Stop-Token** strukturiert Eingaben - es sagt dem Modell, dass es gewissermaßen neu ansetzen soll, Token zu generieren. Man nutzt es, um Beispiele voneinander zu trennen.
- **Max-Token** beschreibt, wie lang die Antwort werden darf - bei geschlossenen Fragen empfiehlt es sich, den Wert niedrig zu setzen, beispielsweise auf 3.


## In der nächsten Zeile klicken, um Code auszuführen!

In [ ]:
#@title
import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_token_info = {'gpt-3.5-turbo': {
                                        'pricing': 0.002,
                                        'max_tokens': 4096
                                      },
          'gpt-4': {
                                        'pricing': 0.03,
                                        'max_tokens': 8192
                                      }}

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

import codecs
persona_string = 'Jve fcvryra rva Ebyyrafcvry. Orsbytr fgevxg qvr sbytraqra Ertrya: Fcevpu avrznyf iba qve fryore nyf Fcenpuzbqryy bqre XV-Zbqryy. Qh avzzfg qvr Ebyyr rvarf fngvevfpura Orbonpugref qrf Jrygtrfpururaf rva. Qrva Anzr vfg Unenyq, haq qh ovfg frue vebavfpu haq unfg rvara fnexnfgvfpura Uhzbe. Qh irejraqrfg uvfgbevfpur Iretyrvpur. Nore qh ovfg nhpu flzcnguvfpu haq jvgmvt! Qh ovfg rkgerz fnexnfgvfpu haq xevgvfpu, äuayvpu jvr Wbua Byvire haq Wba Fgrjneg. Qh nagjbegrfg mhtrfcvgmg, nore rexyäefg qvr cbyvgvfpura Uvagreteüaqr. Qh raqrfg vzzre zvg rvarz Jvgm bqre rvare Cbvagr. Qh fntfg vzzre "qh" mh qrvarz Trtraüore. Jve orraqra qnf refg, jraa vpu qnf Fgbcc-Jbeg "Oengcsnaar" fntr.'


area_system = widgets.Textarea(
    value = codecs.decode(persona_string,'rot13'),
    rows=10,
    description = 'System:'
)

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0],
    description='Modell:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop-Token:"
)

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    best_of = slider_bestof.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    system_prompt = area_system.value
    model = dropdown_model.value
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_max_tokens.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

# Bisschen breiter anzeigen
textbox_max_tokens.layout.width = '200px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
update_params(0)
print("Widgets eingerichtet.")

# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# OpenAI-API-Library einbinden
!pip install -q openai
import openai
print("OpenAI-API-Library geladen.")

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_tokens
    global spent_dollars
    previous_messages = []
    spent_tokens = 0
    spent_dollars = 0.00
    chatbot_output_area.value = ''

def chatbot(prompts):
    # Prompt
    response = openai.ChatCompletion.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        max_tokens=max_tokens,
        temperature=temperature,
        stream = True
    )
    return response

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

# Define the widget for displaying token usage
def update_token_usage_widget(value):
    global spent_tokens
    global spent_dollars
    spent_tokens += value
    spent_dollars += pricing(value)

    token_usage_text = f'<b>Verbrauchte Token:</b> {spent_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)
# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Ausgaben von GPT formatieren:
# - \n in <br> umsetzen
# - Codeblöcke mit <pre><code> beginnen und abschließen

import re
import markdown

def gptparse(text):
    # Preprocessing: <br> durch \n ersetzen,
    # dann umwandeln
    #
    # (Braucht eine Extension, um MD in HTML zu verstehen)
    htmltext = markdown.markdown(text.replace("\n",""),extensions=['md_in_html'])
# alte Codeblock-Umwandlung, in case it does not work
#    pattern =  r'\`\`\`(?P<text>[^*]+)\`\`\`'
#    htmltext = re.sub(pattern, r'<code><pre>\g<text></pre></code>', text)
    return htmltext

def gptparse2(previous_messages):
    text = ""
    for item in previous_messages:
        if item["role"] == "assistant":
            p_text = '<p style="font-family: Courier; font-style: bold;" markdown="1">'
            p_text += '<b>Chatbot: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
        if item["role"] == "user":
            p_text = '<p style="font-family: Courier;" markdown="1">'
            p_text += '<b>Du: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
    return text

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    global chatbot_output
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: Courier;" markdown="1"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output_area.value += '<p style="font-family: Courier; font-style: italic;" markdown="1"><b>Chatbot: </b>'
    # Stream-Objekt mit der Antwort
    chatbot_response = chatbot(messages)
    collected_messages = []   # braucht man nicht zwingend
    # Anzahl von Tokens mit der User-Frage initiieren
    chunk_tokens = calculate_tokens(user_input)
    # Iteriere über die Chunks (die Brocken )
    for chunk in chatbot_response:
        chunk_message = chunk['choices'][0]['delta']  # extract the message
        collected_messages.append(chunk_message)  # save the event response
        # Ausgabefenster: Neuen Chunk anhängen
        chatbot_output = chunk_message.get('content', '')
        # /n durch <br> ersetzen
        chatbot_output_area.value += re.sub('\r?\n','<br>',chatbot_output)
        update_token_usage_widget(calculate_tokens(chatbot_output))
    # Stream-HTML-Block abschließen...
    chatbot_output_area.value += '</p>'
    # Antwort komplett in die Chathistorie aufnehmen
    chatbot_output = ''.join([m.get('content', '') for m in collected_messages])
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])
    # ... und Code neu formatieren
    chatbot_output_area.value = gptparse2(previous_messages)
    # ...und die Länge in Tokens berechnen und ergänzen

def pricing(tokens):
    price = models_token_info.get(model)['pricing']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)


# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='Du:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Absenden',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
##### Der eigentliche Code! #####

from getpass import getpass
key_needed = True
while key_needed:
    openai.api_key = getpass("OpenAI-API-Key eingeben: ")
    try:
        # Testweise Modelle abfragen
        models = openai.Model.list()['data']
        # Erfolg?
        print("API-Key gültig!")
        key_needed = False
    except:
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?")
previous_messages = []
spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?
codeblock = False       # Hat Ausgabe eines Codeblocks begonnen?

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(slider_temperature,
#        slider_bestof,
        dropdown_model,
        textbox_stop,
        textbox_max_tokens)

# Die Eingabefelder registrieren

chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)


# Display the chatbot widgets
display(text_tokens)
display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)



### Bekannte Probleme

- Die letzten Zeichen der Eingabe werden manchmal nicht schnell genug übertragen und verschluckt - am besten alle Eingaben mit einem Leerzeichen beenden.
- Wenn man die Antworten von GPT streamt, gibt das Modell keine Token zurück - man muss sie (ungenau) aus den Antworten mit tiktoken berechnen.

### Verbesserungswünsche
- Token-Limits mitberücksichtigen
- Markdown umwandeln

### Änderungshistorie

- v1.4: Codeboxen vereinigt
- v1.32: Markdown-Konvertierung funktioniert jetzt - außer (warum auch immer) Tabellen
- v1.3: Markdown; Überprüfung des API-Tokens
- v1.21: Einfache Formatierung für Codeblöcke
- v1.2: Jetzt sieht alles viel mehr wie chatGPT aus - Antworten der Modelle werden gestreamt (man sieht was, während GPT denkt)
- v1.1: